| Name        | ID |
|--------------|----|
| Tigist Wondimneh | UGR/2538/12 |
| Yared Tsegaye | UGR/8284/12 | 
| Yeabsira Driba | UGR/4951/12 |

## Background Discussion

### Overview of SqueezeNet

**SqueezeNet** is a compact convolutional neural network (CNN) architecture that was introduced with the primary goal of reducing model size while retaining comparable accuracy to larger models like AlexNet. This architecture is particularly beneficial for deployment in environments where computational resources and storage are limited, such as mobile devices or embedded systems.



### Motivation and Design Principles

The development of SqueezeNet was motivated by the need for more efficient neural network architectures that could operate within the constraints of low-power and low-memory devices. The key design principles behind SqueezeNet include:

- **Parameter Reduction:** One of the main goals was to decrease the number of parameters drastically. This was achieved by employing smaller convolution filters and reducing the input channel depth using squeeze layers.
- **Maintaining Accuracy:** Despite the reduction in parameters, maintaining a high level of accuracy comparable to AlexNet was crucial. This was addressed through innovative architectural decisions like the use of Fire modules.
- **Incremental Refinement:** The architecture allows for flexible scaling of the model’s width (number of channels) and resolution of the input data, providing a means to balance between accuracy and model size.



### Key Components

- **Fire Module:** At the heart of SqueezeNet is the Fire module, which consists of a squeeze layer (1x1 convolutions) followed by an expand layer that has a mix of 1x1 and 3x3 convolutions. This design significantly reduces the parameter count while allowing the network to expand its capacity to capture complex features.
- **Delayed Downsampling:** SqueezeNet strategically delays downsampling to deeper layers in the network, allowing for larger activation maps in the initial layers. This helps in maintaining high classification accuracy with fewer parameters.



### Comparisons with Predecessors

- **AlexNet:** While AlexNet utilized large convolution filters in its first layer (11x11 filters), SqueezeNet uses much smaller filters throughout the network, leading to a drastic reduction in parameters. Both architectures aim for high accuracy on ImageNet, but SqueezeNet is much smaller in size.
- **Innovations Over AlexNet:** Unlike AlexNet, which uses a straightforward sequence of convolutional layers followed by fully connected layers, SqueezeNet introduces modular design through its Fire modules and eliminates most of the fully connected layers, which are parameter-heavy.


The introduction of SqueezeNet marked a significant step forward in neural network design, pushing the boundaries of efficiency in deep learning architectures. It opened up new possibilities for AI applications in resource-constrained environments and influenced subsequent developments in network design aimed at reducing model size while maintaining performance.


# **SqueezeNet Model**

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

The below code implements the SqueezeNet architecture using TensorFlow and Keras, which is known for its efficiency by using Fire modules that combine squeezed and expanded convolution layers to reduce parameter count while maintaining performance. The model processes images with an initial size of 227×227×3227×227×3, includes several convolutional, pooling, and dropout layers, and concludes with a global average pooling and a softmax activation for classification into 1000 classes. This makes it ideal for resource-constrained environments needing high-performance image classification.

In [ ]:
def fire_module(x, squeeze_planes, expand_planes):
    squeeze = tf.keras.layers.Conv2D(squeeze_planes, (1, 1), activation='relu', padding='same')(x)
    expand1 = tf.keras.layers.Conv2D(expand_planes, (1, 1), activation='relu', padding='same')(squeeze)
    expand3 = tf.keras.layers.Conv2D(expand_planes, (3, 3), activation='relu', padding='same')(squeeze)
    return tf.keras.layers.concatenate([expand1, expand3])

def SqueezeNet(input_shape=(227, 227, 3), num_classes=1000):
    img_input = tf.keras.Input(shape=input_shape)
    
    x = tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(img_input)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze_planes=16, expand_planes=64)
    x = fire_module(x, squeeze_planes=16, expand_planes=64)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze_planes=32, expand_planes=128)
    x = fire_module(x, squeeze_planes=32, expand_planes=128)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = fire_module(x, squeeze_planes=48, expand_planes=192)
    x = fire_module(x, squeeze_planes=48, expand_planes=192)
    x = fire_module(x, squeeze_planes=64, expand_planes=256)
    x = fire_module(x, squeeze_planes=64, expand_planes=256)
    
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(num_classes, (1, 1), padding='same', activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Activation('softmax')(x)
    
    model = tf.keras.models.Model(inputs=img_input, outputs=x)
    return model

# **Data Preparation**

The following code includes functions for preprocessing images by scaling their pixel values to [-1, 1] and for loading and labeling images from a directory structure, resizing them to 227x227 for use with SqueezeNet. This setup is ideal for preparing image datasets for neural network training.

In [ ]:
def preprocess_input_squeezenet(img):
    # Scale pixel values to the range [-1, 1]
    img /= 255.0
    img -= 0.5
    img *= 2.0
    return img

def load_images_and_labels(base_path, image_size=(227, 227)):
    data = []
    labels = []
    classes = sorted(os.listdir(base_path))

    for label, cls in enumerate(classes):
        cls_folder = os.path.join(base_path, cls)
        for img_name in os.listdir(cls_folder):
            img_path = os.path.join(cls_folder, img_name)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
            img = tf.keras.preprocessing.image.img_to_array(img)
            img = preprocess_input_squeezenet(img)
            data.append(img)
            labels.append(label)

    data = np.array(data, dtype="float32")
    labels = np.array(labels)
    return data, labels, classes

In [ ]:
X_train, y_train, classes = load_images_and_labels('./data/Alzheimer_s Dataset/train')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1765, random_state=42)

In [ ]:
X_test, y_test, classes = load_images_and_labels('./data/Alzheimer_s Dataset/test')

# **Model Training**

In [ ]:
model = SqueezeNet(input_shape=(227, 227, 3), num_classes=len(classes))  # len(classes) should be the number of classes in your dataset

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Data augmentation generator
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to file
tflite_model_path = "squeezenet_sign_language.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print(f"Model saved as TensorFlow Lite model at {tflite_model_path}")